In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path

from preprocessing.logParser import openLogs

In [2]:
BASE_DIR = os.path.abspath('test_statistics.ipynb')
BASE_DIR = os.path.dirname(BASE_DIR)
BASE_DIR = BASE_DIR + '/spbu_ACADRU_spring_2018-TL'
cursor = openLogs(BASE_DIR)

Описания таблиц и полей:
https://docs.google.com/document/d/12AQyASm_m0lV-aJQNmTCzIbfO2hj49unslcWMaxXjWY/edit# 

problem_check
cursor.executescript("""
        DROP TABLE IF EXISTS problem_check;
        CREATE TABLE problem_check 
        (
            user_id INT
            ,course_name varchar(300) 
            ,execute_time varchar(100)
            ,page varchar(300)
            ,questions varchar(2000)
            ,grade real
            ,max_grade real
        )
    """)


     problem_events = ["edx.problem.hint.demandhint_displayed", "edx.problem.hint.feedback_displayed", "problem_check",
                      "problem_check_fail", "problem_graded", "problem_rescore", "problem_rescore_fail",
                      "problem_reset", "problem_save", "problem_show", "reset_problem", "reset_problem_fail",
                      "save_problem_fail", "save_problem_success", "showanswer"]
                      
https://docs.google.com/document/d/12AQyASm_m0lV-aJQNmTCzIbfO2hj49unslcWMaxXjWY/edit# 

In [3]:
#and questions NOT LIKE '__' means that we needn't questions like {}
#and attempts < 4 means that we use only three first attempts, jthers attempts are not interesting
request = """
        select 
        user_id, attempts, questions,
        page, grade, max_grade
        from problem_check
        where course_name = "course-v1:spbu+ACADRU+spring_2018"
        and questions NOT LIKE '__'
        and attempts < 4
        """.format("https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/b434df438c2045eba63afc089d64b3de/244a81ebf8a64b609cd8587bbe18edc5/")
cursor.execute(request)
data = cursor.fetchall()

columns_names = []
for i in cursor.description[:]:
    columns_names.append(i[0])

df = pd.DataFrame(data=data, columns=columns_names)

OperationalError: no such column: attempts

### модуль тестов, пункт 1 a: (средний результат за тест)
подумать, что с page 

In [ ]:
listForAverage=pd.DataFrame(columns=('page','questions','attempts','grade','max_grade'))

In [ ]:
listForAverage=df.sort_values(by=['questions','attempts'])

In [ ]:
print(listForAverage)

In [ ]:
average=pd.DataFrame(columns=('page','test','attempt','average_grade','max_grade'))

In [ ]:
currentTest=''
for row in listForAverage.itertuples():
    if not currentTest:
        currentTest=row.questions
        currentAttempt=row.attempts
        gradeCounter=0
        numOfSolutions=0
        
    if currentTest==row.questions:
        if currentAttempt==row.attempts:
            gradeCounter=gradeCounter+row.grade
            numOfSolutions=numOfSolutions+1
        else:
            average=average.append({'page':row.page,'test':currentTest,'attempt':currentAttempt,'average_grade':gradeCounter/numOfSolutions,'max_grade':row.max_grade},ignore_index=True)            
            gradeCounter=0
            numOfSolutions=0
            currentAttempt=row.attempts
    else:
        average=average.append({'page':row.page,'test':currentTest,'attempt':currentAttempt,'average_grade':gradeCounter/numOfSolutions,'max_grade':row.max_grade},ignore_index=True)            
        gradeCounter=0
        numOfSolutions=0
        currentAttempt=row.attempts
        currentTest=row.questions     

In [ ]:
print(average)

### модуль тестов, пункт 1 с: (процент правильных ответов на вопрос)

In [ ]:
#parsing of questions string in two columns: question and result
listForQuestions=pd.DataFrame(columns=('question','attempt','user_id','result'))

for rowCounter in range(df.shape[0]):
    currentAttempt=df.attempts[rowCounter]
    currentUser=df.user_id[rowCounter]
    row=(df.questions[rowCounter][1:-1]).split(',')
    for cellRow in row:
        tmprow=cellRow.split(':')
        if tmprow[1]=='True':
            listForQuestions=listForQuestions.append({'question':tmprow[0],'attempt':currentAttempt,'user_id':currentUser,'result':1}, ignore_index=True)
        else:
            listForQuestions=listForQuestions.append({'question':tmprow[0],'attempt':currentAttempt,'user_id':currentUser,'result':0}, ignore_index=True)


In [ ]:
listForQuestions['question'] = listForQuestions['question'].map(str.strip)
listForQuestions=listForQuestions.sort_values(by=['question','attempt','user_id'])

In [ ]:
percentage=pd.DataFrame(columns=('question','attempt','numOfAnswers','percentageOfRightAnswers'))

In [ ]:
currentQuestion=''
for row in listForQuestions.itertuples():
    if not currentQuestion:
        currentQuestion=row.question
        currentAttempt=row.attempt
        tmpUsersList=pd.DataFrame(columns=('user_id','result'))
    
    prevAttempt=currentAttempt
    currentAttempt=row.attempt   
    prevQuestion=currentQuestion
    currentQuestion=row.question   
    
    if prevQuestion==currentQuestion :        
        if prevAttempt==currentAttempt :
            if currentAttempt==1:
                tmpUsersList=tmpUsersList.append({'user_id':row.user_id,'result':row.result}, ignore_index=True)
            else:
                for user in tmpUsersList:
                    if user==row.user_id:
                        user.result=row.result #updating result for new attempt
        else:
            percentage=percentage.append({'question':prevQuestion,'attempt':prevAttempt,'numOfAnswers':tmpUsersList.shape[0],'percentageOfRightAnswers':100*(((tmpUsersList.result).sum())/tmpUsersList.shape[0])},ignore_index=True)            
    else:        
        percentage=percentage.append({'question':prevQuestion,'attempt':prevAttempt,'numOfAnswers':tmpUsersList.shape[0],'percentageOfRightAnswers':100*(((tmpUsersList.result).sum())/tmpUsersList.shape[0])},ignore_index=True)
        tmpUsersList=pd.DataFrame(columns=('user_id','result')) 
        tmpUsersList=tmpUsersList.append({'user_id':row.user_id,'result':row.result}, ignore_index=True)

In [ ]:
print(percentage)